## Задание

- Собрать информацию о всех строящихся объектах на сайте "наш.дом.рф"
- Cохранить ее в pandas dataframe
- Cохранить pandas dataframe в excel
- Cохранить pandas dataframe в pickle
- Cохранить pandas dataframe в БД

### Импорт библиотек

In [1]:
import requests
import pandas as pd
import xlsxwriter
from sqlalchemy import create_engine

In [2]:
pd.options.display.max_columns = 200
pd.options.display.max_rows = 30
pd.options.display.precision = 2
pd.options.display.show_dimensions = True

### Список объектов

In [3]:
limit_ = 1000
all_obj_ids = []
for offset_ in range(0, 12000, 1000):
    url = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/kn/object?offset={offset_}&limit={limit_}&sortField=devId.devShortCleanNm&sortType=asc&objStatus=0'
    response = requests.get(url)
    objects_data = response.json()
    objects_list = objects_data.get('data').get('list')
    objids = [x.get('objId') for x in objects_list]
    all_obj_ids.extend(objids)

In [4]:
len(all_obj_ids) # количество объектов

10527

### Данные по объекту

In [5]:
all_objects_description = []

for id in all_obj_ids:
    url = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/object/{id}'
    response_object = requests.get(url)
    obj = response_object.json()
    all_objects_description.append(obj)

### Сохранение в DataFrame

In [6]:
df = pd.DataFrame.from_dict(pd.json_normalize(all_objects_description), orient='columns')


In [8]:
df.shape

(10527, 109)

### Сохранение в формат Excel

In [11]:
writer = pd.ExcelWriter("data/house.xlsx", engine = 'xlsxwriter')
df.to_excel(writer)
writer.save()

### Сохранение в формат Pickle

In [12]:
df.to_pickle('data/house.pkl')

### Сохранение в БД

In [17]:
engine = create_engine('sqlite:///data/house.db', echo=True)
sqlite_connection = engine.connect()

In [18]:
df_sql = df.copy()

In [19]:
df_sql = df.applymap(str)

In [20]:
sqlite_table = "data/dom_rf"
df_sql.to_sql(sqlite_table, sqlite_connection, if_exists='append')

2022-08-15 21:42:15,635 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("data/dom_rf")
2022-08-15 21:42:15,636 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-08-15 21:42:15,636 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("data/dom_rf")
2022-08-15 21:42:15,637 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-08-15 21:42:15,642 INFO sqlalchemy.engine.Engine 
CREATE TABLE "data/dom_rf" (
	"index" BIGINT, 
	errcode TEXT, 
	"data.id" TEXT, 
	"data.pdId" TEXT, 
	"data.developer.devId" TEXT, 
	"data.developer.devShortCleanNm" TEXT, 
	"data.developer.devShortNm" TEXT, 
	"data.developer.devFullCleanNm" TEXT, 
	"data.developer.problObjCnt" TEXT, 
	"data.developer.buildObjCnt" TEXT, 
	"data.developer.comissObjCnt" TEXT, 
	"data.developer.regRegionDesc" TEXT, 
	"data.developer.devPhoneNum" TEXT, 
	"data.developer.devSite" TEXT, 
	"data.developer.devEmail" TEXT, 
	"data.developer.devInn" TEXT, 
	"data.developer.devOgrn" TEXT, 
	"data.developer.devKpp" TEXT, 
	"data.developer.devLegalA

10527